## Import

In [9]:
import pandas as pd
import numpy as np
from sklearn.model_selection import StratifiedKFold  # 계층적 K-Fold
from sklearn.metrics import f1_score
from sklearn.linear_model import LogisticRegression  # baseline 모델
import re

## Data Load

In [10]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

- 일단 feature를 아주 한정적으로 사용하고, 1만 출력하는 코드보다 나은지 비교할 것.

### re_registration / major_data
- object / bool type을 int type으로

In [11]:
# 1. 매핑 딕셔너리 생성
# '아니요'는 0(신규), '예'는 1(재등록)로 설정합니다.
re_reg_map = {'아니요': 0, '예': 1}

# 2. Train 및 Test 데이터 변환
train['re_registration'] = train['re_registration'].map(re_reg_map)
test['re_registration'] = test['re_registration'].map(re_reg_map)

# 3. 데이터 타입 정수형(int) 확정
# 결측치가 발생할 경우를 대비해 fillna(0)를 포함하는 것이 안전합니다.
train['re_registration'] = train['re_registration'].fillna(0).astype(int)
test['re_registration'] = test['re_registration'].fillna(0).astype(int)

# major_data 컬럼을 bool형에서 정수형으로 변환
train['major_data'] = train['major_data'].astype(int)

### interested_company 전처리

In [12]:
def process_interested_company(df):
    
    def encode_company(x):
        # 1. 결측치(NaN) 처리 -> 0
        if pd.isna(x):
            return 0
        
        # 문자열로 변환 및 앞뒤 공백 제거
        x = str(x).strip()
        
        # 2. 빈 문자열인 경우 -> 0
        if len(x) == 0:
            return 0

        # 3. 부정적인 키워드('없', '아직', '모르')가 포함되어 있으면 -> 0
        # 'nan' 문자열도 결측치 취급
        negative_keywords = ['없', '아직', '모르', 'nan']
        if any(keyword in x for keyword in negative_keywords):
            return 0
            
        # 4. 한 글자 응답인 경우 -> 0
        if len(x) == 1:
            return 0
            
        # 5. 특수문자만 있는 경우 (한글, 영어, 숫자가 하나도 없는 경우) -> 0
        # 정규표현식: [a-zA-Z0-9가-힣] 하나라도 있으면 True, 없으면 False
        if not re.search('[a-zA-Z0-9가-힣]', x):
            return 0
        
        # 6. 그 외의 경우 (관심 기업 있음) -> 1
        return 1

    # 변환 함수 적용
    df['interested_company'] = df['interested_company'].apply(encode_company)
    
    return df

# 전처리 실행
train = process_interested_company(train)
test = process_interested_company(test)

### 이전에 몇 번 수강했는지 나타내는 prev_attendance_count 만들기

In [ ]:
def create_experience_feature(df):
    # 1. 대상 컬럼 리스트 정의
    prev_cols = [f'previous_class_{i}' for i in range(3, 9)]
    
    # 2. 누적 참여 횟수 계산
    # '없음'이 아니면서 결측치도 아닌 경우를 카운트
    df['prev_attendance_count'] = df[prev_cols].apply(
        lambda x: (x != '해당없음') & (x.notna()), axis=1
    ).sum(axis=1)
    
    return df

# Train 및 Test 적용
train = create_experience_feature(train)
test = create_experience_feature(test)

In [ ]:
class_cols = ['class1', 'class2', 'class3', 'class4']
train['class_count'] = train[class_cols].notna().sum(axis=1)
test['class_count'] = test[class_cols].notna().sum(axis=1)

In [ ]:
def preprocess_categorical_col(train_df, test_df, col_name, threshold=10):
    """
    1) 빈도가 threshold 미만인 항목을 '기타'로 통합
    2) train+test에 동일하게 적용
    3) 이후 one-hot 더미 컬럼을 생성하기 위한 준비 단계 (category로 바꾸지 않음)
    """
    # 1. Train 기준 빈도
    counts = train_df[col_name].value_counts()
    common_vals = counts[counts >= threshold].index.tolist()
    
    def group_rare(x):
        if x in common_vals:
            return x
        return '기타'
    
    train_df[col_name] = train_df[col_name].apply(group_rare)
    test_df[col_name]  = test_df[col_name].apply(group_rare)
    
    # 여기서는 category로 바꾸지 않고 문자열/객체로 둔다.
    # (One-Hot 시 자동으로 처리됨)
    return train_df, test_df